In [ ]:
import os 
# os.environ["GPU"] = "1"
# os.environ["CLANG"] = "1"
import numpy as np
import tqdm

from tinygrad.tensor import Tensor
from tinygrad.nn import optim
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt    

In [ ]:
class Model():
    def __init__(self) -> None:
        self.l1 = Tensor.glorot_uniform(1, 2)
        self.mid = Tensor.glorot_uniform(2, 2)
        self.l2 = Tensor.glorot_uniform(2, 1)

    def __call__(self, x):
        x = x/3.1415
        x = (x.dot(self.l1)).tanh()
        x = (x.dot(self.mid)).tanh()
        x = (x.dot(self.l2))
        return x

    def save(self, filename):
        with open(filename+'.npz', 'wb') as f:
            np.savez(f, *[p.cpu().numpy() for p in optim.get_parameters(self)])

    def load(self, filename):
        try:
            arrays = np.load(filename)
            params = [Tensor(arrays[k]) for k in arrays.keys()]
            for param, load in zip(optim.get_parameters(self), params):
                if param.shape == load.shape:
                    param = load
                else:
                    raise f"Shape mismatch!  {param.shape} != {load.shape}"
            print("successfully loaded")
        except Exception as e:
            print("failed to load")
            print(e)

    def __repr__(self) -> str:
        rep = ""
        for param in optim.get_parameters(self):
            rep += f"{param.shape} {param.flatten().detach().numpy()}\n"
        return rep


In [ ]:
def sin_gen():
    while True:
        x = np.random.random(1)*2*np.pi - np.pi
        y = np.sin(x)+ np.random.randn(1)*0.001
        yield x, y

def sin_eval():
    while True:
        x = np.random.random(1)*2*np.pi - np.pi
        y = np.sin(x)
        yield x, y

In [ ]:
# plot the dict
x,y = [],[]
x__, y__ = [ ], [ ]
for i in range(1000):
    x_, y_ = next(sin_gen())
    x___, y___ = next(sin_eval())
    x__.append(x___)
    y__.append(y___)
    x.append(x_)
    y.append(y_)

plt.scatter(x, y, label = "train", color = "blue", alpha=0.5)
plt.scatter(x__, y__, label = "eval", color = "red", alpha=0.5, marker='x')

In [ ]:
def lr_exp_decay(lr0, epoch, decay):
    return max(lr0 * np.exp(-decay*epoch),0.00001)

# graph
lrs = [lr_exp_decay(0.01, i, 0.05) for i in range(50)]
plt.plot(lrs)
plt.show()

In [ ]:
myModel = Model()
save_name = "sin"
myOptim = optim.Adam(optim.get_parameters(myModel), lr=0.01)

def MSE(y, y_):
    return ((y - y_)**2).sum()  

def MAE(y, y_):
    return (y - y_).abs().mean()

loss_fxn = MAE
# loss_fxn = MSE
print(myModel)

In [ ]:
# train loop
total_preds = []
train_history = []
val_history = []
best = 0
patience = 0
best_val = np.inf
X_train, Y_train = [], []
epochs = 2
# main loop
for e in tqdm.tqdm(range(epochs)):
    myOptim.lr = lr_exp_decay(0.03, e, 0.05)
    X_train = np.array(X_train, dtype=np.float32)
    Y_train = np.array(Y_train, dtype=np.float32)
    # train loop
    Tensor.training = True
    total_loss = 0
    for i in range(100):
        x, y = next(sin_gen())
        x = Tensor(np.array(x,dtype=np.float32), requires_grad=False)
        y = Tensor(np.array(y,dtype=np.float32), requires_grad=False)
        myOptim.zero_grad()
        y_ = myModel(x)[0]
        loss = loss_fxn(y, y_)
        total_loss += loss.detach().numpy()/100
        loss.backward()
        myOptim.step()
    train_history.append(total_loss)
    print("Train loss", total_loss)

    # validation loop
    Tensor.training = False
    total_val_loss = 0
    temp=[]
    for i in range(100):
        x_val, y_val = next(sin_eval())
        x_val = Tensor(np.array(x_val,dtype=np.float32), requires_grad=False)
        y_val = Tensor(np.array(y_val,dtype=np.float32), requires_grad=False)
        y_val_ = myModel(x_val)[0]
        temp.append((x_val.numpy(), y_val_.numpy()))
        val_loss = loss_fxn(y_val, y_val_)
        total_val_loss += val_loss.detach().numpy()/100
    val_history.append(total_val_loss)
    print("Validation loss", total_val_loss)

    if total_val_loss < best_val:
        best_val = total_val_loss
        myModel.save(save_name)
        print("-----saved-----")


In [ ]:
plt.plot(train_history, label="train")
plt.plot(val_history, label="val")
plt.legend()

In [ ]:
#  load the best model
myModel.load(f"{save_name}.npz")
print(myModel)

In [ ]:
# plot distribution of weights per layer
plt.figure(figsize=(10,10))
for i in range(len(optim.get_parameters(myModel))):
    plt.subplot(len(optim.get_parameters(myModel)),1, i+1)
    plt.hist(optim.get_parameters(myModel)[i].detach().numpy().flatten(), label=f"layer {i}")
    plt.legend()
plt.show()

In [ ]:
#  start the codegen process
import subprocess
#  generate the code DEBUG=5
cmd2 = f'CLANG=1 OPT=1 python3 compile.py --name={save_name}'
res2 = subprocess.run(cmd2, shell=True, capture_output=True, text=True)
print(res2.stdout)
C_name = res2.stdout.split(":")[-1].strip()
print("C name is ",C_name)
# save as .so
so_name = C_name[:-2] + ".so"
print("so name is ",so_name)

In [ ]:
compile_cmd = f'gcc -shared -lm -fPIC -x c -o {so_name} {C_name}' # --rtlib=compiler-rt
print("COMILING....\n",compile_cmd)
res3 = subprocess.run(compile_cmd, shell=True, capture_output=True, text=True)
if len(res3.stderr) > 0:
    print("ERROR", res3.stderr)
else:
    print("SUCCESS", res3.stdout)

In [ ]:
import ctypes
my_model = ctypes.CDLL("./"+so_name)
my_model.forward.argtypes = [ctypes.c_float]
my_model.forward.restype = ctypes.c_float

#  test 
y = my_model.forward(0.5)
print(y)

In [ ]:
#  evaluate the model
X_eval, Y_eval, Y_pred, C_pred  = [], [], [], []
x = np.linspace(-np.pi, np.pi, 50, dtype=np.float32)
for i in range(50):
    x_, y_ = next(sin_eval())
    X_eval.append(x_)
    Y_eval.append(y_)
    out = myModel(Tensor(np.array(x_, dtype=np.float32), requires_grad=False)).numpy()
    Y_pred.append(out)
    res = my_model.forward(ctypes.c_float(x[i]))
    C_pred.append(res)

X_eval = np.array(X_eval, dtype=np.float32)
Y_eval = np.array(Y_eval, dtype=np.float32)
Y_pred = np.array(Y_pred, dtype=np.float32)
C_pred = np.array(C_pred, dtype=np.float32)

plt.scatter(X_eval, Y_eval, label = "eval", color = "red", alpha=0.5, marker='o')
plt.scatter(X_eval, Y_pred, label = "pred", color = "blue", alpha=0.3, marker='x')
plt.scatter(x, C_pred, label = "C", color = "green", alpha=1, marker='+')
plt.legend()
plt.show()